# 导入所需包

In [3]:
import numpy as np
import pandas as pd
import os
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# 导入数据集

In [5]:
train_labels = pd.read_csv('train_labels.csv')
train_sequences = pd.read_csv('train_sequences.csv')
test_sequences = pd.read_csv('test_sequences.csv')
valid_labels = pd.read_csv('validation_labels.csv')
valid_sequences = pd.read_csv('validation_sequences.csv')
submission = pd.read_csv('sample_submission.csv')

# 数据预处理

In [6]:
# train_labels['ID']: 获取ID列中的值
# .str: 将Series对象转为字符串以应用字符串方法
# .rsplit('_', n=1): 从字符串右侧开始，以下划线为分隔符，n=1表示分割一次（例如"RNA123_456"变成["RNA123", "456"]）
# .str[0]: 获取分割后的第一个元素
# 这行代码创建了新列'target'，用于存储ID中下划线前的部分，这部分通常是RNA的名称(target_id 是目标 RNA 序列的唯一标识符, 后面的是残基编号)
train_labels['target'] = train_labels['ID'].str.rsplit('_', n=1).str[0]
valid_labels['target'] = valid_labels['ID'].str.rsplit('_', n=1).str[0]

# train_sequences.merge(): 合并两个DataFrame
# how='left': 执行左连接，保留左表所有行
# left_on='target_id': 使用左表(train_sequences)的'target_id'列进行匹配
# right_on='target': 使用右表(train_labels)的'target'列进行匹配
# 这行代码将RNA序列信息与其对应的标签连接起来
# 左表的target_id和右表的target同样是rna的名称，列名不同
train = train_sequences.merge(train_labels, how='left', left_on='target_id', right_on='target')
validation = valid_sequences.merge(valid_labels, how='left', left_on='target_id', right_on='target')

# 日期转成从1970年1月1日开始的毫秒数(训练集和测试集的时间戳)
# pd.to_datetime(): 将字符串转换为日期时间格式
# .astype('int64'): 将日期时间转换为整数格式（毫秒数）
# 这行代码将'temporal_cutoff'列转换为整数格式，以便进行数值计算
train['temporal_cutoff'] = pd.to_datetime(train['temporal_cutoff']).astype('int64') 
test_sequences['temporal_cutoff'] = pd.to_datetime(test_sequences['temporal_cutoff']).astype('int64')
validation['temporal_cutoff'] = pd.to_datetime(validation['temporal_cutoff']).astype('int64')

In [7]:
submission['target'] = submission['ID'].str.rsplit('_', n=1).str[0]  # 提取目标RNA名称
test = test_sequences.merge(submission, how='left', left_on='target_id', right_on='target')

In [8]:
# 生成序列的长度
train['seq_length'] = train['sequence'].str.len()
test['seq_length'] = test['sequence'].str.len()
validation['seq_length'] = validation['sequence'].str.len()

# 特征补充
对于局部的特定结构可能会有相似的位置关系，比如"ACC"这三个连着的碱基，它们可能都是有相似的相对关系，中间的C在上面旁边的AC在下面。所以这里新加特征: 前两个碱基、后两个碱基，有利于模型更精准捕捉这种局部的位置特征，效果会变好

In [9]:
# 生成前后残基，按RNA分组再shift

# 创建包含所有数据集的列表，使代码可以一次性处理所有数据框
dataframes = [train, validation, test]

# 定义一个字典，指定要添加的上下文特征及其偏移量
# 正值表示向前移动（获取前面的残基）
# 负值表示向后移动（获取后面的残基）
shifts = {
    'prev_res': 1,       # 前一个残基
    'next_res': -1,      # 后一个残基
    'prev_two_res': 2,   # 前两个残基
    'next_two_res': -2   # 后两个残基
}

# 为每个数据框添加所有残基特征
# df.groupby('target_id'):
# groupby函数将数据框按'target_id'列分组
# 这确保shift操作只在同一RNA序列内进行，不会跨不同RNA序列
# 对于每个unique的'target_id'值，都创建一个独立的数据子集

# 列选择 ['resname']
# 从分组后的数据中选择'resname'列
# 此列包含RNA残基的核苷酸类型(如A、C、G、U)

# 数据偏移 .shift(shift_value)
# shift函数在序列中移动数据位置
# 例如shift(1)会将每个值向下移动一行，获取前一个值
# shift(-1)则向上移动一行，获取后一个值
# 关键点：shift在每个分组内独立进行，保证了上下文信息的准确性

# 将操作结果保存到新列中，列名为字典中的键
# 例如创建'prev_res'列存储前一个残基信息
for df in dataframes:
    for col_name, shift_value in shifts.items():
        df[col_name] = df.groupby('target_id')['resname'].shift(shift_value)

# 可选：填充边界值的NaN
# 序列边界处会产生缺失值：
# 序列开头的残基没有"前一个残基"
# 序列结尾的残基没有"后一个残基"
# 使用'None'字符串填充这些边界缺失值
for df in dataframes:
    for col in shifts.keys():
        df[col] = df[col].fillna('None')  # 或使用其他合适的填充值

In [ ]:
# 核苷酸组成特征（碱基组成特征）
def add_nucleotide_composition(df):
    # 计算序列中GC含量(G+C碱基占总长度的比例)
    df['GC_content'] = df['sequence'].apply(lambda x: (x.count('G') + x.count('C'))/len(x))
    # 计算序列中AU含量(A+U碱基占总长度的比例)
    df['AU_content'] = df['sequence'].apply(lambda x: (x.count('A') + x.count('U'))/len(x))
    # 以及单独计算G、A、C、U的比例
    df['G_content'] = df['sequence'].apply(lambda x: x.count('G')/len(x))
    df['A_content'] = df['sequence'].apply(lambda x: x.count('A')/len(x))
    df['C_content'] = df['sequence'].apply(lambda x: x.count('C')/len(x))
    df['U_content'] = df['sequence'].apply(lambda x: x.count('U')/len(x))
    return df

In [ ]:
# 局部结构复杂度
def add_local_complexity(df):
    def calc_local_complexity(seq, window=5):
        # seq: RNA序列字符串
        # window: 窗口大小参数，默认为5（表示考虑当前位置周围共11个碱基）
        result = []   # 创建一个空列表，用于存储序列中每个位置的局部复杂度值
        for i in range(len(seq)):
            start = max(0, i-window)   # 计算窗口的起始位置，确保不越界
            end = min(len(seq), i+window+1)   # 计算窗口的结束位置，确保不越界

            # 提取子序列：获取局部窗口范围内的RNA子序列
            # 结果：得到以当前位置为中心的局部序列片段
            local_seq = seq[start:end]

            # 计算局部GC含量
            # 原理：GC含量是RNA结构稳定性的重要指标，GC碱基对形成3个氢键，比AU碱基对(2个氢键)更稳定
            # 生物学意义：GC含量高的区域通常形成更稳定的结构
            local_gc = (local_seq.count('G') + local_seq.count('C'))/len(local_seq)
            result.append(local_gc)
        return result
    
    df['local_complexity'] = df['sequence'].apply(calc_local_complexity)
    return df

In [ ]:
# 堆叠能量特征
def add_stacking_energy(df):   # 定义堆叠能量参数 (kcal/mol)
    # 字典定义：创建包含所有16种可能二核苷酸堆叠能量值的字典
    # 数据来源：这些参数基于实验测定的热力学数据
    # 单位：千卡/摩尔(kcal/mol)，负值表示稳定相互作用
    # 物理意义：数值越小（越负）表示堆叠作用越稳定，如CC(-3.36)比AA(-0.93)更稳定
    stacking_energy = {
        'AA': -0.93, 'AC': -2.24, 'AG': -1.30, 'AU': -1.10,
        'CA': -2.11, 'CC': -3.36, 'CG': -2.36, 'CU': -2.08,
        'GA': -1.41, 'GC': -2.44, 'GG': -1.53, 'GU': -1.80,
        'UA': -0.90, 'UC': -1.70, 'UG': -1.80, 'UU': -0.80
    }
    
    def calc_avg_stacking(seq):
        if len(seq) <= 1:
            return 0
        energy = 0   # 累计总堆叠能量，初始为0
        pairs = 0    # 计数有效的二核苷酸对，初始为0
        
        # 遍历序列中的每个二核苷酸对（长度为2的子序列）
        for i in range(len(seq)-1):
            dinuc = seq[i:i+2]
            if dinuc in stacking_energy:
                energy += stacking_energy[dinuc]
                pairs += 1
        # 计算平均堆叠能量：将总能量除以有效的二核苷酸对数
        return energy/pairs if pairs > 0 else 0
    
    df['avg_stacking_energy'] = df['sequence'].apply(calc_avg_stacking)
    return df

In [2]:
# 结构模块特征
def add_structural_motifs(df):
    # 常见的RNA结构模块模式:

    # tetraloop_GNRA：
    # 格式：G + 任意核苷酸 + 嘌呤(A或G) + A
    # 功能：极其稳定的发夹环结构，常促进长程相互作用
    # 生物学意义：在核糖体RNA中高度保守，影响RNA整体折叠

    # tetraloop_UNCG
    # 格式：U + 任意核苷酸 + C + G
    # 功能：热力学上非常稳定的四核苷酸环
    # 生物学意义：能够稳定相邻的RNA结构元素

    # kink_turn：
    # 格式：GA + 两个任意核苷酸 + A + 任意核苷酸 + A
    # 功能：导致RNA主链形成典型的"扭折"结构
    # 生物学意义：常见于复杂RNA结构的转角处，造成约120°的转弯

    # C_loop：
    # 格式：CC + 任意两核苷酸(NN) + 5-7个任意核苷酸 + GG
    # 功能：形成特殊的RNA内部环状结构
    # 生物学意义：常与蛋白结合，参与RNA-蛋白互作
    motifs = {
        'tetraloop_GNRA': r'G[AGUC][AG]A',   
        'tetraloop_UNCG': r'U[AGUC]CG',       
        'kink_turn': r'GA[AGUC]{2}A[AGUC]A',  
        'C_loop': r'CC[AGUC]{2}[AGUC]{5,7}GG' # 可简化为CC[AGUC]{7,9}GG, 但保留结构语义
    }
    
    import re
    for motif_name, pattern in motifs.items():
        # 模式匹配：
        # re.search(pattern, x)：在序列x中搜索模式pattern
        # 返回值：找到匹配返回1，未找到返回0
        df[f'has_{motif_name}'] = df['sequence'].apply(
            lambda x: 1 if re.search(pattern, x) else 0)
    
    return df

In [14]:
# 应用特征工程
for df in [train, validation, test]:
    df = add_nucleotide_composition(df)  # 添加核苷酸组成特征
    df = add_local_complexity(df)  # 添加局部结构复杂度特征
    df = add_stacking_energy(df)   # 添加堆叠能量特征
    df = add_structural_motifs(df)  # 添加结构模块特征

In [16]:
# 显示处理后的validation数据前5行，用于验证数据处理是否正确
# train.head()
print(train.columns)

Index(['target_id', 'sequence', 'temporal_cutoff', 'description',
       'all_sequences', 'ID', 'resname', 'resid', 'x_1', 'y_1', 'z_1',
       'target', 'seq_length', 'prev_res', 'next_res', 'prev_two_res',
       'next_two_res', 'GC_content', 'AU_content', 'G_content', 'A_content',
       'C_content', 'U_content', 'local_complexity', 'avg_stacking_energy',
       'has_tetraloop_GNRA', 'has_tetraloop_UNCG', 'has_kink_turn',
       'has_C_loop'],
      dtype='object')


In [17]:
categorical_columns = ['ID', 'target_id', 'description','all_sequences','target','resname',
                       'prev_res','next_res','prev_two_res','next_two_res','GC_content', 'AU_content', 
                       'G_content', 'A_content','C_content', 'U_content', 'local_complexity', 'avg_stacking_energy',
                       'has_tetraloop_GNRA', 'has_tetraloop_UNCG', 'has_kink_turn','has_C_loop']

# 文字编码处理

In [18]:
# 从sklearn.preprocessing模块导入LabelEncoder类，LabelEncoder用于将分类变量转换为数值形式（0, 1, 2...）
from sklearn.preprocessing import LabelEncoder

# 初始化一个空字典，用于存储每个分类特征的编码器
label_encoders = {}

for col in categorical_columns:
    # train[col].astype(str): 将列值转换为字符串类型
    # .fillna('NA'): 用'NA'填充缺失值
    train[col] = train[col].astype(str).fillna('NA')

    # 创建编码器实例
    le = LabelEncoder()
    # 训练编码器并将分类值转换为数值形式
    train[col] = le.fit_transform(train[col])
    # 将编码器保存到字典中，以列名为键
    label_encoders[col] = le

for col in categorical_columns:
    # 与训练集类似的操作，但应用于测试数据
    # test[col].astype(str).fillna('missing'): 将列转为字符串并用'missing'填充缺失值
    test[col] = test[col].astype(str).fillna('missing')
    
    le = LabelEncoder()
    test[col] = le.fit_transform(test[col])
    label_encoders[col] = le

# 对验证集的分类变量进行编码
for col in categorical_columns:
    # 与训练集类似的操作，但应用于验证数据
    validation[col] = validation[col].astype(str).fillna('none')
    
    le = LabelEncoder()
    validation[col] = le.fit_transform(validation[col])
    label_encoders[col] = le
# 注意：
# 字典只会显示LabelEncoder对象的引用，而不会显示实际的编码映射或转换后的数值。这是Python对象默认的字符串表示形式导致的。

# 定义训练集和训练目标

In [ ]:
# 这行代码定义了将用于模型训练的特征变量列表，包含以下特征：
features = ['temporal_cutoff', 'resname', 'resid', 'seq_length', 'target_id','ID','description','all_sequences',
            'prev_res','next_res','prev_two_res','next_two_res','GC_content', 'AU_content', 
                       'G_content', 'A_content','C_content', 'U_content', 'local_complexity', 'avg_stacking_energy',
                       'has_tetraloop_GNRA', 'has_tetraloop_UNCG', 'has_kink_turn','has_C_loop']

# 这行定义了模型的预测目标：这是一个包含3个坐标的列表，表示三维空间中的点
coord_targets = ['x_1','y_1','z_1']

# 配置训练模型

In [20]:
# 这段代码是训练三个机器学习模型，分别预测空间中的x、y、z三个坐标值
models = {}   # 创建一个空字典，用来存放训练好的模型，字典的键将是坐标名('x_1'、'y_1'、'z_1')，值是对应的训练好的模型
for coord in coord_targets: 
    # 这个循环会依次处理'x_1'、'y_1'、'z_1'三个坐标
    print(f'Training model for {coord}...')
    model = xgb.XGBRegressor(             # XGBRegressor是一种强大的机器学习模型，适合预测数值（比如坐标值）
        objective='reg:squarederror',     # 使用均方误差作为优化目标（让预测值尽量接近真实值）
        n_estimators=1000,                 # 模型会建立500棵小树，它们一起工作做预测（想象成500个专家一起投票）
        max_depth=7,                      # 每棵树最多有5层（决定了模型的复杂度）
        learning_rate=0.1,                # 学习步长，值小时学习更谨慎但需要更多树（类似于人学习新知识时的谨慎程度）
        subsample=0.8,                    # 随机取样比例：每棵树随机使用60%的训练数据（防止过度拟合）
        colsample_bytree=0.8,             # 列采样比例：每棵树随机使用60%的特征（增加模型多样性）
        # min_child_weight=3,               # 控制过拟合的参数，值越大越保守（类似于人学习时的谨慎程度）
    )
    
    mean_val = train[coord].mean()        # 计算当前坐标的均值，用于填充缺失值（如果有的话）
    valid_idx = train[coord].notna()      # 找出坐标不是缺失值(列不为空)的行（创建了一个布尔数组），只用它们训练
    train[coord].fillna(mean_val, inplace=True)   # 用平均值填充缺失的坐标值，但注意这里填充后实际并没有使用这些填充的值

    # model.fit(...)：训练模型
    # train[valid_idx][features]：只用非缺失值的行，选取特征列作为输入
    # train[valid_idx][coord]：对应的坐标值作为预测目标
    model.fit(train[valid_idx][features], train[valid_idx][coord])

    models[coord] = model                 # 将训练好的模型保存到字典中，以坐标名称为键

# 这段代码的关键是它为每个坐标（x、y、z）创建了一个单独的预测模型，并且只使用有效数据（非缺失值）来训练模型

# # 导入计算均方误差的函数
# from sklearn.metrics import mean_squared_error

# highest = -1  # 初始TM-score设置为-1，表示还没有计算过
# N_ESTIMATORS = [1500, 1000, 500]  # 定义不同的树的数量
# MAX_DEPTH = [5, 9, 7]  # 定义不同的树的深度
# LEARNING_RATE = [0.01, 0.1, 0.2]  # 定义不同的学习率
# SUBSAMPLE = [0.6, 0.7, 0.8]  # 采样比例保持不变
# COLSAMPLE_BYTREE = [0.6, 0.7, 0.8]  # 列采样比例保持不变
# MIN_CHILD_WEIGHT = [1, 3, 5]  # 控制过拟合
# BEST_PARAMS = {}
# num = 1  # 用于输出目前的参数组合编号

# for a in N_ESTIMATORS:
#     for b in MAX_DEPTH:
#         for c in LEARNING_RATE:
#             for d in SUBSAMPLE:
#                 for e in COLSAMPLE_BYTREE:
#                     for f in MIN_CHILD_WEIGHT:
#                         print(f'正在测试参数组合 {num}: n_estimators={a}, max_depth={b}, learning_rate={c}, subsample={d}, colsample_bytree={e}, min_child_weight={f}')
#                         num += 1
#                         # 创建模型实例
#                         models = {}  # 初始化一个空字典，用于存储每个坐标的模型
#                         for coord in coord_targets: 
#                             model = xgb.XGBRegressor(
#                                 objective='reg:squarederror',
#                                 n_estimators = a,
#                                 max_depth = b,
#                                 learning_rate = c,
#                                 subsample = d,
#                                 colsample_bytree = e,
#                                 min_child_weight = f
#                             )
                            
#                             # 训练模型
#                             valid_idx = train[coord].notna()
#                             model.fit(train[valid_idx][features], train[valid_idx][coord])
#                             models[coord] = model
                        
#                         validation_predictions = validation[['temporal_cutoff', 'resname', 'resid', 'seq_length', 'target_id','ID','description','all_sequences',
#                         'prev_res','next_res','prev_two_res','next_two_res']]
#                         for coord in coord_targets:
#                             # 这是一个循环，遍历coord_targets列表（包含'x_1', 'y_1', 'z_1'）
#                             # 对于每个坐标（例如'x_1'）：
#                             # 使用之前训练好的对应模型（models[coord]）进行预测
#                             # 预测使用的是validation[features]中的特征数据
#                             # 将预测的结果存储在validation_predictions数据框的对应坐标列中
#                             validation_predictions[coord] = models[coord].predict(validation[features])

#                         tmp = calculate_tm_score(validation_predictions, validation)  # 计算验证集的TM-score

#                         if tmp > highest:
#                             highest = tmp
#                             BEST_PARAMS = {
#                                 'n_estimators': a,
#                                 'max_depth': b,
#                                 'learning_rate': c,
#                                 'subsample': d,
#                                 'colsample_bytree': e,
#                                 'min_child_weight': f
#                             }

# print(f'最佳参数组合: {BEST_PARAMS}', f' TM-score: {highest:.20f}')

Training model for x_1...
Training model for y_1...
Training model for z_1...


# TM-score
是一个重要的指标，用于评估预测的RNA结构与真实结构之间的相似性。TM-score值范围在0到1之间，值越高表示预测结果越接近真实结构。

一般来说，TM-score大于0.5表示预测结果较好，大于0.7表示非常好。

In [ ]:
# 计算当前rna目标序列d0值
# seq_len: RNA序列的长度
# 函数根据序列长度返回不同的d0值
def calculate_d0(seq_len):
    if seq_len > 30:
        # 如果序列长度大于30，使用公式计算：0.6*(seq_len-0.5)**0.5-2.5
        # 这是一个平方根函数，随着序列变长，d0值会增加但增速会减慢
        return 0.6*(seq_len-0.5)**0.5-2.5
    else:
        # 当序列长度≤30时，根据不同长度范围返回固定值：
        if seq_len < 12:
            return 0.3
        if seq_len < 16:
            return 0.4
        if seq_len < 20:
            return 0.5
        if seq_len < 24:
            return 0.6
        else:   
            return 0.7
# 这种分段设计表明d0值与RNA序列长度有关，对于预测算法或评估可能很重要
# 随着序列长度增加，d0值也相应增加，可能反映了更长序列需要更灵活的评估标准

In [22]:
# 计算当前rna目标序列di值
def calculate_dis(x1, y1, z1, x2, y2, z2):
    return ((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)**0.5

In [23]:
# calculate_tm_score函数计算预测RNA结构与目标结构之间的TM-score（Template Modeling score）
# 这是评估RNA结构预测准确性的重要指标
# pred: 包含预测坐标的数据框
# target: 包含目标（真实）坐标的数据框
def calculate_tm_score(pred, target):
    tm_score = []   # 初始化一个空列表tm_score存储每个RNA序列的TM-score
    cur_id = ''     # 初始化cur_id为空字符串，用于跟踪当前处理的RNA序列ID
    for i in range(len(pred)):
        # 如果遇到新序列就把上一个序列tm-score存起来，并重置其他记录
        if pred['target_id'][i] != cur_id:   # 当遇到新序列时（当前行的target_id与之前不同）：
            if cur_id != '':
                # 如果不是第一个序列（cur_id不为空），将上一个序列的累计得分除以序列长度，得到TM-score并保存
                tm_score.append(cur_id_score/seq_len)
            seq_len = pred['seq_length'][i]     # 获取当前序列的长度
            d0=calculate_d0(seq_len)            # 根据序列长度计算d0值（使用上一个函数中的calculate_d0）
            # 计算当前行（残基位置）的得分：1/(1+(距离²/d0²))
            cur_id_score = (1/(1+(calculate_dis(pred['x_1'][i],pred['y_1'][i],pred['z_1'][i],target['x_1'][i],target['y_1'][i],target['z_1'][i]))**2/d0**2))
            cur_id = pred['target_id'][i]       # 更新当前处理的序列ID
            
        # 特殊处理最后一个样本，取所有tm-score均值作为输出
        # 如果是数据集的最后一行：添加最后一个序列的得分，然后返回所有序列TM-score的平均值
        # 否则：累加当前残基位置的得分
        else :
            if i == len(pred)-1:
                tm_score.append(cur_id_score/seq_len)
                return np.mean(tm_score)
            else:
                cur_id_score += (1/(1+(calculate_dis(pred['x_1'][i],pred['y_1'][i],pred['z_1'][i],target['x_1'][i],target['y_1'][i],target['z_1'][i]))**2/d0**2))

# 代入验证集

In [24]:
validation = validation[['temporal_cutoff', 'resname', 'resid', 'seq_length', 'target_id','ID','description','all_sequences',
                         'prev_res','next_res','prev_two_res','next_two_res','GC_content', 'AU_content', 
                       'G_content', 'A_content','C_content', 'U_content', 'local_complexity', 'avg_stacking_energy',
                       'has_tetraloop_GNRA', 'has_tetraloop_UNCG', 'has_kink_turn','has_C_loop'] +['x_1','y_1','z_1']]
validation.head()

,temporal_cutoff,resname,resid,seq_length,target_id,ID,description,all_sequences,prev_res,next_res,...,U_content,local_complexity,avg_stacking_energy,has_tetraloop_GNRA,has_tetraloop_UNCG,has_kink_turn,has_C_loop,x_1,y_1,z_1
0,1653696000000000000,2,1,69,0,0,7,2,3,2,...,1,9,9,0,1,0,0,-5.499,8.520000,8.605000
1,1653696000000000000,2,2,69,0,11,7,2,2,2,...,1,9,9,0,1,0,0,-5.826,10.453000,14.010000
2,1653696000000000000,2,3,69,0,22,7,2,2,2,...,1,9,9,0,1,0,0,-5.849,14.768000,17.584999
3,1653696000000000000,2,4,69,0,33,7,2,2,2,...,1,9,9,0,1,0,0,-5.784,19.985001,18.666000
4,1653696000000000000,2,5,69,0,44,7,2,2,1,...,1,9,9,0,1,0,0,-5.755,25.533001,17.132999


In [25]:
valid_prediction = validation.copy()
for coord in coord_targets:
    valid_prediction[coord] = models[coord].predict(validation[features])
valid_prediction.head()

,temporal_cutoff,resname,resid,seq_length,target_id,ID,description,all_sequences,prev_res,next_res,...,U_content,local_complexity,avg_stacking_energy,has_tetraloop_GNRA,has_tetraloop_UNCG,has_kink_turn,has_C_loop,x_1,y_1,z_1
0,1653696000000000000,2,1,69,0,0,7,2,3,2,...,1,9,9,0,1,0,0,-10.562522,89.101929,111.511246
1,1653696000000000000,2,2,69,0,11,7,2,2,2,...,1,9,9,0,1,0,0,-10.662208,86.972404,107.534691
2,1653696000000000000,2,3,69,0,22,7,2,2,2,...,1,9,9,0,1,0,0,-14.013841,85.700874,102.770615
3,1653696000000000000,2,4,69,0,33,7,2,2,2,...,1,9,9,0,1,0,0,-13.291576,82.225822,102.823044
4,1653696000000000000,2,5,69,0,44,7,2,2,1,...,1,9,9,0,1,0,0,-13.366677,80.006676,96.059631


In [40]:
calculate_tm_score(validation, valid_prediction)

0.001318134876687638

# 代入测试集

In [28]:
# 这行代码从test数据框中选择了特定的列，并将结果重新赋值给test变量，这意味着测试数据集现在只保留了这些指定的列
test = test[['temporal_cutoff', 'resname', 'resid', 'seq_length', 'target_id','ID','description','all_sequences',
             'prev_res','next_res','prev_two_res','next_two_res','GC_content', 'AU_content', 
                       'G_content', 'A_content','C_content', 'U_content', 'local_complexity', 'avg_stacking_energy',
                       'has_tetraloop_GNRA', 'has_tetraloop_UNCG', 'has_kink_turn','has_C_loop']]

In [29]:
# 创建test数据框的完整副本，并将其赋值给test_predictions
# 这样做的目的是保留原始测试数据，同时在副本上添加预测结果
test_predictions = test.copy()
for coord in coord_targets:
    test_predictions[coord] = models[coord].predict(test[features])
    
# 展示部分预测
print("Test predictions sample:")
test_predictions[['ID', 'resname', 'resid', 'x_1', 'y_1', 'z_1']].head()

Test predictions sample:


,ID,resname,resid,x_1,y_1,z_1
0,0,2,1,-10.562522,89.101929,111.511246
1,11,2,2,-10.662208,86.972404,107.534691
2,22,2,3,-14.013841,85.700874,102.770615
3,33,2,4,-13.291576,82.225822,102.823044
4,44,2,5,-13.366677,80.006676,96.059631


In [30]:
# 将已预测的第1个原子位置('x_1', 'y_1', 'z_1')的坐标值复制给第2到第5个原子
# 通过循环遍历2到5的数字(range(2, 6))，创建并填充新的坐标列
# 执行后，test_predictions数据框将包含以下新列：

# 'x_2', 'x_3', 'x_4', 'x_5' (值均与'x_1'相同)
# 'y_2', 'y_3', 'y_4', 'y_5' (值均与'y_1'相同)
# 'z_2', 'z_3', 'z_4', 'z_5' (值均与'z_1'相同)

# 在RNA结构预测中，通常每个核苷酸有多个原子点需要预测
# 完整的解决方案应该单独预测每个原子的位置
# 这种简化方法显然不能准确反映真实RNA分子中各原子的相对位置，只是一个临时性解决方案。
for i in range(2, 6):
    test_predictions[f'x_{i}'] = test_predictions['x_1']
    test_predictions[f'y_{i}'] = test_predictions['y_1']
    test_predictions[f'z_{i}'] = test_predictions['z_1']

In [31]:
submission[['x_1','y_1','z_1','x_2','y_2','z_2','x_3','y_3','z_3','x_4','y_4','z_4','x_5','y_5','z_5']] = test_predictions[['x_1','y_1','z_1','x_2','y_2','z_2','x_3','y_3','z_3','x_4','y_4','z_4','x_5','y_5','z_5']] 

In [32]:
submission.to_csv('submission.csv', index=False)